# Specify Paths 

In [ ]:
import os
import pandas as pd
import sys

sys.path.append("/home/ndrakes1/force_estimation/preprocess/fpgav3-data-collection/unit_convert") ## set path to unit convert


# --- SETUP PATHS ---
fpga_v3_data_collection_path = os.path.expanduser("~/force_estimation/preprocess/fpgav3-data-collection/")
true_preprocess_path = os.path.expanduser("~/force_estimation/preprocess/")
unit_convert_path = fpga_v3_data_collection_path + "unit_convert/unit_convert.py"
si_unit_json_path = os.path.join(fpga_v3_data_collection_path, "unit_convert/sawRobotIO1394-PSM1-334809.xml.json")

train_name_path_csv = "/home/ndrakes1/force_estimation/data/dvrk_si_train/teleop_train_dvrk_si.csv"
val_test_name_path_csv = "/home/ndrakes1/force_estimation/data/dvrk_si_train/teleop_val_test_dvrk_si.csv"

train_name_path_unit_convert_csv = "/home/ndrakes1/force_estimation/data/dvrk_si_train/teleop_train_dvrk_si_unitConvert.csv"
val_test_name_path_unit_convert_csv = "/home/ndrakes1/force_estimation/data/dvrk_si_train/teleop_val_test_dvrk_si_unitConvert.csv"

path_to_data_folder = os.path.expanduser("/home/ndrakes1/force_estimation/data/dvrk_si_train/")
new_preprocessed_data_folder = "preprocess"
new_dir = os.path.join(path_to_data_folder, new_preprocessed_data_folder)

train_sub = os.path.join(new_dir, "train/joints/")
val_sub = os.path.join(new_dir, "val/joints/")
test_sub = os.path.join(new_dir, "test/joints/")

## paths for high freq preprocessing

In [21]:
preproces_path = "/home/ndrakes1/force_estimation/preprocess/force-estimation-fpga-data-preprocess"
preprocessing_script_path = preproces_path + "/preprocessing.py"
split_val_test_path = preproces_path + "/split_val_test.py"

preprocessing_train_cmd = "python3 " + preprocessing_script_path + " " + train_name_path_unit_convert_csv
preprocessing_val_test_cmd ="python3 " + preprocessing_script_path + " " + val_test_name_path_unit_convert_csv

print(preprocessing_train_cmd)

python3 /home/ndrakes1/force_estimation/preprocess/force-estimation-fpga-data-preprocess/preprocessing.py /home/ndrakes1/force_estimation/data/dvrk_si_train/teleop_train_dvrk_si_unitConvert.csv


In [22]:
os.makedirs(train_sub, exist_ok=True)
os.makedirs(val_sub, exist_ok=True)
os.makedirs(test_sub, exist_ok=True)

# Unit Conversion to SI units

In [23]:
sys.path.append(os.path.join(fpga_v3_data_collection_path, "unit_convert"))
from unit_convert import unitConvert, saveConfigFile

# TRAIN
train_converted = unitConvert(si_unit_json_path, train_name_path_csv)
saveConfigFile(train_name_path_csv, train_converted)  # auto-saves *_unitConvert.csv

# VAL/TEST
val_converted = unitConvert(si_unit_json_path, val_test_name_path_csv)
saveConfigFile(val_test_name_path_csv, val_converted)

Data written to /home/ndrakes1/force_estimation/data/dvrk_si_train/teleop_train_dvrk_si_unitConvert.csv
Data written to /home/ndrakes1/force_estimation/data/dvrk_si_train/teleop_val_test_dvrk_si_unitConvert.csv


# High Freq Preprocessing

In [24]:
sys.path.append(os.path.join(true_preprocess_path, "force-estimation-fpga-data-preprocess"))

from preprocessing import preprocess_csv
from split_val_test import split_val_test

# Preprocess TRAIN
train_df = preprocess_csv(train_name_path_unit_convert_csv)
train_df.to_csv(os.path.join(train_sub, "interpolated_all_joints.csv"), index=False)

# Preprocess VAL/TEST
val_test_df = preprocess_csv(val_test_name_path_unit_convert_csv)
interpolated_all_joints_path = os.path.join(true_preprocess_path, "interpolated_all_joints.csv")
val_test_df.to_csv(interpolated_all_joints_path, index=False)

# --- SPLIT VAL / TEST ---
split_val_test(interpolated_all_joints_path,split_ratio=.5)


/home/ndrakes1/force_estimation/preprocess/force-estimation-fpga-data-preprocess/split_val_test.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_csv, header=None)


Validation set saved to val.csv 519558 rows
Test set saved to test.csv 519559 rows


## preprocessing 